# Bedrock AgentCore Gateway を使用して OpenAPI API を MCP ツールに変換する

## 概要
お客様は JSON または YAML 形式の OpenAPI 仕様を持ち込み、Bedrock AgentCore Gateway を使用して API を MCP ツールに変換できます。NASA のオープン API を API キーを使用して呼び出す火星天気エージェントの構築を実演します。

Gateway ワークフローでは、エージェントを外部ツールに接続するために以下の手順が含まれます：
* **Gateway 用のツールを作成する** - REST API 用の OpenAPI 仕様などのスキーマを使用してツールを定義します。OpenAPI 仕様は Amazon Bedrock AgentCore によって解析され、Gateway が作成されます。
* **Gateway エンドポイントを作成する** - インバウンド認証を備えた MCP エントリポイントとして機能する gateway を作成します。
* **Gateway にターゲットを追加する** - gateway がリクエストを特定のツールにルーティングする方法を定義する OpenAPI ターゲットを設定します。OpenAPI ファイルの一部であるすべての API は MCP 互換のツールになり、Gateway エンドポイント URL を通じて利用可能になります。各 OpenAPI Gateway ターゲットのアウトバウンド認証を設定します。
* **エージェントコードを更新する** - 統合された MCP インターフェースを通じてすべての設定済みツールにアクセスするために、エージェントを Gateway エンドポイントに接続します。

![仕組み](images/openapi-gateway-apikey.png)

### チュートリアルの詳細


| 情報                 | 詳細                                                     |
|:---------------------|:----------------------------------------------------------|
| チュートリアルタイプ | インタラクティブ                                         |
| AgentCore コンポーネント | AgentCore Gateway, AgentCore Identity                     |
| エージェントフレームワーク | Strands Agents                                            |
| Gateway ターゲットタイプ | OpenAPI                                                   |
| エージェント         | 火星天気エージェント                                     |
| インバウンド認証 IdP | Amazon Cognito                                            |
| アウトバウンド認証   | API キー                                                  |
| LLM モデル           | Anthropic Claude Sonnet 3.7, Amazon Nova Pro              |
| チュートリアルコンポーネント | AgentCore Gateway の作成と AgentCore Gateway の呼び出し   |
| チュートリアル分野   | 分野横断的                                               |
| 例の複雑さ           | 簡単                                                     |
| 使用 SDK             | boto3                                                     |

チュートリアルの最初のパートでは、いくつかの AmazonCore Gateway ターゲットを作成します

### チュートリアルのアーキテクチャ
このチュートリアルでは、OpenAPI yaml/json ファイルで定義された操作を MCP ツールに変換し、Bedrock AgentCore Gateway でホストします。
デモンストレーションのために、火星の天気に関するクエリに答える火星天気エージェントを構築します。このエージェントは NASA のオープン API を使用します。このソリューションは Amazon Bedrock モデルを使用する Strands Agent を使用しています。
この例では、火星の天気のための getInsightWeather ツールを持つ非常にシンプルなエージェントを使用します。

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Jupyter notebook（Python カーネル）
* uv
* AWS 認証情報
* Amazon Cognito

In [ ]:
# 最新の botocore と boto3 ライブラリをダウンロードしていることを確認してください。
import shutil
import subprocess
import sys

def ensure_uv_installed():
    if shutil.which("uv") is None:
        print("🔧 'uv' not found. Installing with pip...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "uv"])
    else:
        print("✅ 'uv' is already installed.")

def uv_install(*packages):
    ensure_uv_installed()
    uv_path = shutil.which("uv")
    print(f"📦 Installing {', '.join(packages)} using uv...")
    subprocess.check_call([uv_path, "pip", "install", *packages])

uv_install("botocore", "boto3")

In [ ]:
# Amazon SageMaker ノートブックを使用していない場合は AWS 認証情報を設定する
import os
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_PROFILE'] = 'cline2'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [ ]:
import os
import sys

# 現在のスクリプトのディレクトリを取得する
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # __file__ が定義されていない場合のフォールバック（例：Jupyter）

# utils.py を含むディレクトリ（一つ上のレベル）に移動する
utils_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# sys.path に追加する
sys.path.insert(0, utils_dir)

# 今、あなたは utils をインポートできます
import utils

In [ ]:
#### Gateway が引き受ける IAM ロールを作成する
import utils

agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Gateway へのインバウンド認証のための Amazon Cognito プールの作成

In [ ]:
# Cognito ユーザープールの作成
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# Get discovery URL

cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

# ゲートウェイの作成

In [ ]:
# CMK なしで Cognito オーソライザーを使用して CreateGateway を行います。前のステップで作成した Cognito ユーザープールを使用します
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # クライアントは Cognito で設定された ClientId と一致する必要があります。例： 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='DemoGWOpenAPIAPIKeyNasaOAI',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # IAM ロールは Gateway の作成・一覧表示・取得・削除の権限を持っている必要があります
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with OpenAPI target'
)
print(create_response)
# GatewayTarget 作成に使用される GatewayID を取得する
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

# NASA オープン API を Bedrock AgentCore Gateway を使用して MCP ツールに変換する

Japanese translation of the title "Transforming NASA Open APIs into MCP tools using Bedrock AgentCore Gateway".

私たちは NASA のオープン API から気象データを取得する Mars Weather エージェントを作成します。NASA Insight API に[こちら](https://api.nasa.gov/)で登録する必要があります。登録は無料です！登録すると、メールで API キーが届きます。その API キーを使用して、OpenAPI ターゲットを作成するための認証情報プロバイダーを設定してください。

In [ ]:
import boto3
from pprint import pprint
from botocore.config import Config

acps = boto3.client(service_name="bedrock-agentcore-control")

response=acps.create_api_key_credential_provider(
    name="NasaInsightAPIKey", 
    apiKey="", # api.nasa.gov でサインアップして API キーを取得してください。メールで API キーを取得するのにかかる時間は 2 分です。
)

pprint(response)
credentialProviderARN = response['credentialProviderArn']
pprint(f"Egress Credentials provider ARN, {credentialProviderARN}")

# OpenAPI ターゲットの作成

#### NASA Open API の json ファイルを S3 にアップロードする

In [ ]:
# S3 クライアントを作成する
session = boto3.session.Session()
s3_client = session.client('s3')
sts_client = session.client('sts')

# AWS アカウント ID とリージョンの取得
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
# パラメータを定義する
bucket_name = '' # OpenAPI json ファイルをアップロードする s3 バケット。
file_path = 'openapi-specs/nasa_mars_insights_openapi.json'
object_key = 'nasa_mars_insights_openapi.json'
# ファイルを put_object を使用してアップロードし、レスポンスを読み取る
try:
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=file_data)

    # アップロードされたオブジェクトの ARN をアカウント ID とリージョンで構築する
    openapi_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'Uploaded object S3 URI: {openapi_s3_uri}')
except Exception as e:
    print(f'Error uploading file: {e}')

#### 送信認証を設定し、ゲートウェイターゲットを作成する

In [ ]:
# S3 Uri for OpenAPI 仕様ファイル
nasa_openapi_s3_target_config = {
    "mcp": {
          "openApiSchema": {
              "s3": {
                  "uri": openapi_s3_uri
              }
          }
      }
}

# API キー 認証情報プロバイダーの設定
api_key_credential_config = [
    {
        "credentialProviderType" : "API_KEY", 
        "credentialProvider": {
            "apiKeyCredentialProvider": {
                    "credentialParameterName": "api_key", # API プロバイダーが期待する API キー名に置き換えてください。ヘッダーでトークンを渡す場合は "Authorization" を使用してください
                    "providerArn": credentialProviderARN,
                    "credentialLocation":"QUERY_PARAMETER", # API キーの場所。指定可能な値は "HEADER" と "QUERY_PARAMETER" です。
            }
        }
    }
  ]

targetname='DemoOpenAPITargetS3NasaMars'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='OpenAPI Target with S3Uri using SDK',
    targetConfiguration=nasa_openapi_s3_target_config,
    credentialProviderConfigurations=api_key_credential_config)

# Strands エージェントから Bedrock AgentCore Gateway を呼び出す

Strands エージェントは、Model Context Protocol (MCP) 仕様を実装する Bedrock AgentCore Gateway を通じて AWS ツールとシームレスに統合されています。この統合により、AI エージェントと AWS サービス間の安全で標準化された通信が可能になります。

その核心部分として、Bedrock AgentCore Gateway は基本的な MCP API である ListTools と InvokeTools を公開するプロトコル準拠のゲートウェイとして機能します。これらの API により、MCP 準拠のクライアントや SDK は、安全で標準化された方法で利用可能なツールを発見し、対話することができます。Strands エージェントが AWS サービスにアクセスする必要がある場合、これらの MCP 標準化されたエンドポイントを使用してゲートウェイと通信します。

ゲートウェイの実装は (MCP 認証仕様)[https://modelcontextprotocol.org/specification/draft/basic/authorization] に厳密に準拠しており、堅牢なセキュリティとアクセス制御を確保しています。これは、Strands エージェントによるすべてのツール呼び出しが認証ステップを経由することを意味し、強力な機能を提供しながらセキュリティを維持します。

例えば、Strands エージェントが MCP ツールにアクセスする必要がある場合、まず ListTools を呼び出して利用可能なツールを発見し、次に InvokeTools を使用して特定のアクションを実行します。ゲートウェイは必要なすべてのセキュリティ検証、プロトコル変換、サービス連携を処理し、プロセス全体をシームレスかつ安全にします。

このアーキテクチャアプローチにより、MCP 仕様を実装するあらゆるクライアントや SDK がゲートウェイを通じて AWS サービスと対話できるようになり、AI エージェント統合のための汎用性と将来性のあるソリューションとなっています。

In [ ]:
uv_install("mcp[cli]", "strands-agents")

# インバウンド認証のために Amazon Cognito からアクセストークンをリクエストする

In [ ]:
print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes")
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("Token response:", token)

# NASA Open API を使用して Bedrock AgentCore Gateway で火星の天気エージェントに問い合わせる

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## ~/.aws/credentials に設定された IAM グループ/ユーザーは Bedrock モデルへのアクセス権を持っている必要があります
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# ルートの strands ロガーを設定します。問題をデバッグしている場合は、DEBUG に変更してください。
logging.getLogger("strands").setLevel(logging.INFO)

# ログを確認するためのハンドラーを追加する
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # Call the listTools
    tools = client.list_tools_sync()
    # モデルとツールを使用してエージェントを作成する
    agent = Agent(model=yourmodel,tools=tools) ## どのモデルでも好きなものに置き換えることができます
    print(f"Tools loaded in the agent are {agent.tool_names}")
    print(f"Tools configuration in the agent are {agent.tool_config}")
    # エージェントをサンプルプロンプトで呼び出します。これは MCP listTools のみを呼び出し、LLM がアクセスできるツールのリストを取得します。以下は実際にはどのツールも呼び出しません。
    agent("Hi , can you list all tools available to you")
    agent("What is the weather in northern part of the mars")
    # エージェントをサンプルプロンプトで呼び出し、ツールを実行して応答を表示する
    # MCP ツールを明示的に呼び出します。MCP ツール名と引数は、AWS Lambda 関数または OpenAPI/Smithy API と一致する必要があります
    result = client.call_tool_sync(
    tool_use_id="get-insight-weather-1", # あなたはこれを一意の識別子に置き換えることができます。
    name=targetname+"___getInsightWeather", # これは AWS Lambda ターゲットタイプに基づくツール名です。ターゲット名に応じて変更されます
    arguments={"ver": "1.0","feedtype": "json"}
    )
    # ログを確認するためのハンドラーを追加する0
    print(f"Tool Call result: {result['content'][0]['text']}")


**問題: 以下のセルを実行中に下記のエラーが表示される場合、pydantic と pydantic-core のバージョン間に互換性がないことを示しています。**

```
TypeError: model_schema() got an unexpected keyword argument 'generic_origin'
```

**解決方法は？**

互換性のある pydantic==2.7.2 と pydantic-core 2.27.2 を確実にインストールする必要があります。インストール完了後、カーネルを再起動してください。

# クリーンアップ
IAM ロール、IAM ポリシー、認証情報プロバイダー、AWS Lambda 関数、Cognito ユーザープール、S3 バケットなどの追加リソースも作成されており、クリーンアップの一環として手動で削除する必要があるかもしれません。これは実行する例によって異なります。

## ゲートウェイの削除 (オプション)

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)